### Import Libraries

In [1]:
#for web scraping
import requests
from bs4 import BeautifulSoup

#for data cleaning and analysis
import pandas as pd  
import numpy as np
import statsmodels.api as sm

### Webscraping Final Score Data

In [2]:
url = 'https://www.pro-football-reference.com/years/2019/games.htm'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'id': 'games'})
data = pd.read_html(str(table))[0]

### Cleaning and Transforming Data

**Drop unnecessary rows columns**

In [3]:
data.head()

,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,PtsW,PtsL,YdsW,TOW,YdsL,TOL
0,1,Thu,September 5,8:20PM,Green Bay Packers,@,Chicago Bears,boxscore,10,3,213,0,254,1
1,1,Sun,September 8,1:00PM,Tennessee Titans,@,Cleveland Browns,boxscore,43,13,339,0,346,3
2,1,Sun,September 8,1:00PM,Minnesota Vikings,NaN,Atlanta Falcons,boxscore,28,12,269,0,345,3
3,1,Sun,September 8,1:00PM,Philadelphia Eagles,NaN,Washington Redskins,boxscore,32,27,436,0,398,0
4,1,Sun,September 8,1:00PM,Los Angeles Rams,@,Carolina Panthers,boxscore,30,27,349,1,343,3


In [4]:
data.drop([
    'Week','Day','Date','Time','Unnamed: 7','YdsW','TOW','YdsL','TOL'
],axis = 1, inplace = True)

In [5]:
data.rename(index = str, columns = {'Winner/tie':'W'}, inplace = True)
data.rename(index = str, columns = {'Unnamed: 5':'HorA'}, inplace = True)
data.rename(index = str, columns = {'Loser/tie':'L'}, inplace = True)

In [6]:
data = data[data.HorA != 'N'] #drops games on neutral field
data = data[data.W != 'Winner/tie'] #drops weekly header rows

In [7]:
data.dropna(thresh=2, inplace = True) #drops any row with null value in 2nd column

In [8]:
data.head()

,W,HorA,L,PtsW,PtsL
0,Green Bay Packers,@,Chicago Bears,10,3
1,Tennessee Titans,@,Cleveland Browns,43,13
2,Minnesota Vikings,NaN,Atlanta Falcons,28,12
3,Philadelphia Eagles,NaN,Washington Redskins,32,27
4,Los Angeles Rams,@,Carolina Panthers,30,27


**Parse through Winners and Losers to find Home team, Away Team, Home Pts, and Away Pts**

In [9]:
def find_home_away(column1,column2,name):
    row_data = []
    for index, row in data.iterrows():
        c1 = row[column1]
        c2 = row[column2]
        home_or_away = row['HorA']
        if home_or_away == '@':
            row_data.append(c1)
        else:
            row_data.append(c2)
    data[name] = row_data
    return

In [10]:
find_home_away('W','L','Away')
find_home_away('L','W','Home')
find_home_away('PtsW','PtsL','AwayPts')
find_home_away('PtsL','PtsW','HomePts')

In [11]:
data.head()

,W,HorA,L,PtsW,PtsL,Away,Home,AwayPts,HomePts
0,Green Bay Packers,@,Chicago Bears,10,3,Green Bay Packers,Chicago Bears,10,3
1,Tennessee Titans,@,Cleveland Browns,43,13,Tennessee Titans,Cleveland Browns,43,13
2,Minnesota Vikings,NaN,Atlanta Falcons,28,12,Atlanta Falcons,Minnesota Vikings,12,28
3,Philadelphia Eagles,NaN,Washington Redskins,32,27,Washington Redskins,Philadelphia Eagles,27,32
4,Los Angeles Rams,@,Carolina Panthers,30,27,Los Angeles Rams,Carolina Panthers,30,27


**Now no longer need 4 columns**

In [12]:
data.drop([
    'W','HorA','L','PtsW','PtsL'
],axis = 1, inplace = True)

**Then to calculate the Margin**

In [13]:
data['AwayPts'] = data.AwayPts.astype(int)
data['HomePts'] = data.HomePts.astype(int)

In [14]:
data['Margin'] = data['AwayPts'] - data['HomePts']
data.head()

,Away,Home,AwayPts,HomePts,Margin
0,Green Bay Packers,Chicago Bears,10,3,7
1,Tennessee Titans,Cleveland Browns,43,13,30
2,Atlanta Falcons,Minnesota Vikings,12,28,-16
3,Washington Redskins,Philadelphia Eagles,27,32,-5
4,Los Angeles Rams,Carolina Panthers,30,27,3


In [15]:
srs = pd.read_csv('srs.csv')

In [16]:
srs.drop([
    'W','L','T','W-L%','PF','PA','PD','MoV','SoS','OSRS','DSRS'
],axis = 1, inplace = True)

In [17]:
#use dictionary to assign each stat to its respective team
srs_dict = srs.set_index('Tm').stack().groupby(level=0).agg(sum).to_dict()

In [18]:
#create column that contains respective teams stat
data['aSRS'] = data['Away'].map(srs_dict)
data['hSRS'] = data['Home'].map(srs_dict)

In [19]:
data.head()

,Away,Home,AwayPts,HomePts,Margin,aSRS,hSRS
0,Green Bay Packers,Chicago Bears,10,3,7,3.2,-0.9
1,Tennessee Titans,Cleveland Browns,43,13,30,3.4,-1.9
2,Atlanta Falcons,Minnesota Vikings,12,28,-16,-0.1,5.4
3,Washington Redskins,Philadelphia Eagles,27,32,-5,-10.8,0.3
4,Los Angeles Rams,Carolina Panthers,30,27,3,3.9,-7.0


### Linear Regression Model

In [20]:
data['intercept'] = 1 #required to find intercept
x = data[['aSRS','hSRS','intercept']] #dependent variables

reg_model = sm.OLS(data.Margin, x).fit()
reg_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Margin   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     79.25
Date:                Sat, 22 Aug 2020   Prob (F-statistic):           1.16e-27
Time:                        08:42:12   Log-Likelihood:                -1030.2
No. Observations:                 266   AIC:                             2066.
Df Residuals:                     263   BIC:                             2077.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
aSRS           1.0913      0.113      9.625      0.000       0.868       1.315
hSRS          -0.8989      0.110     -8.178      0.000      -1.115      -0.682
intercept      0.1291      0.718      0.180      0.857      -1.285       1.543
==============================================================================
Omnibus:                        1.548   Durbin-Watson:                   2.143
Prob(Omnibus):                  0.461   Jarque-Bera (JB):                1.485
Skew:                           0.089   Prob(JB):                        0.476
Kurtosis:                       2.681   Cond. No.                         6.54
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Spread-Value Calculator**

In [21]:
reg_stats = reg_model.params

In [22]:
ateam = input('Away Team: ')
hteam = input('Home Team: ')
result = round(
    reg_stats['intercept'] + srs_dict[ateam]*reg_stats['aSRS'] + \
    srs_dict[hteam]*reg_stats['hSRS'], 2)
result2 = result*-1
print('The predicted margin is:',result)
print('Therefore, the',ateam,"have a spread of",result2)
print('and the',hteam,'have a spread of',result)

Away Team: Chicago Bears
Home Team: Green Bay Packers
The predicted margin is: -3.73
Therefore, the Chicago Bears have a spread of 3.73
and the Green Bay Packers have a spread of -3.73
